# 🔰PyTorchでニューラルネットワーク基礎 #20【CNNによるテキスト分類】

### 内容
* Qiitaの記事と連動しています
* CNN (Conv2d)を利用したテキスト分類
* 利用するデータ：data/data_60.zip
* data_60.zipは解凍して利用

### データについて
* webサイトへの「ログイン、登録、解約」の３種類の問い合わせを分類する60個データ
* 事前にmecabなどで形態素解析をして単語にIDを割り振り、等長の長さに変換してある
* <bos>文章<eos><pad>の形で長さが11になるまで<pad>を挿入
* カテゴリー数：３
* 単語数：105
* 系列長（文章の長さ）：11


### モデルについて
> 文章（IDベクトル）→埋め込み層→CNN→FC
* 埋め込み層で文章を「系列長ｘ埋め込み次元」の行列に変換
* チャンネル数１を追加してCNNへ入力
* CNNのカーネルサイズに注意。横の長さ＝埋め込み次元。縦の長さ＝単語の前後関係。
* Flatten()で１列に並べてFCへ入力して分類


In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [2]:
#デバイスの選択
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("利用デバイス:", device)

# 精度を計算する関数
def accuracy(y, t):
    _, argmax_list = torch.max(y, dim=1)
    accuracy = sum(argmax_list == t).item()/len(t)
    return accuracy

利用デバイス: cuda


In [3]:
# データの読み込み
# x: IDベクトル
# t: ラベル
# torchテンソルに変換
x = np.loadtxt("data/data_60/x_id_vector.csv", delimiter=",")
t = np.loadtxt("data/data_60/y_label.csv", delimiter=",")
x = torch.LongTensor(x)
t = torch.LongTensor(t)

#--------------------------------------------------------------------------------
# 初期設定
WORDS = 105 # 単語数
SEQ_LEN = 11    # x.shape[1]  # 入力するIDベクトルの長さ
CLASSES = 3 # sum(1 for line in open("data/y_id2tag.txt"))   # 分類数

In [4]:
x, x_test, t, t_test = train_test_split(x,t, stratify=t,  random_state=55)

x = x.to(device)
t = t.to(device)
x_test = x_test.to(device)
t_test = t_test.to(device)

x.shape, x_test.shape

(torch.Size([45, 11]), torch.Size([15, 11]))

### モデルの定義

In [5]:
# モデル定義
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=105, embedding_dim=4)
        self.cnn  = nn.Conv2d(in_channels=1, out_channels=10 ,kernel_size=(2,4))  # チャンネル数，フィルター数，カーネルサイズ kernel_size=(2,4)
        self.flat = nn.Flatten()
        self.fc    = nn.Linear(10*(11-1), 3)
        
    def forward(self, x):
        # idベクトルを(バッチサイズ，高さ(文の単語)，幅(単語の埋め込み表現))のデータに変換
        wv = self.embed(x)
        # (バッチサイズ，チャンネル，高さ(系列長)，幅（埋め込み次元）
        h = wv.reshape(wv.shape[0], 1, wv.shape[1], wv.shape[2])
        h = self.cnn(h)          # 出力 (bs, 10, 10 ,1)
        h = self.flat(h)
        y = self.fc(h)
        return y

model = DNN()
model.to(device)

DNN(
  (embed): Embedding(105, 4)
  (cnn): Conv2d(1, 10, kernel_size=(2, 4), stride=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)

In [6]:
criterion = nn.CrossEntropyLoss()                          # 損失関数：cross_entropy
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)  # 学習率：lr=0.001がデフォルト

In [ ]:
LOOP = 25                               # LOOP : 学習回数
for epoch in range(LOOP):
    optimizer.zero_grad()                # 勾配初期化PyTorchの約束事項
    y = model(x)                         # ネットワークモデルによる予測
    loss = criterion(y, t)               # 誤差計算PyTorchの約束事項
    acc  = accuracy(y,t)                 # 精度の計算
    print(f"{epoch}: loss: {loss.item()}, acc:{acc}")   # 損失と精度の表示
    loss.backward()                      # backward(微分する部分)
    optimizer.step()                     # パラメータ更新PyTorchの約束事項

### 検証してみた

In [238]:
y_test = model(x_test)
acc = accuracy(y_test, t_test)
print("平均精度", acc)

平均精度 0.5333333333333333
